### Best Office Location:

In [6]:
import pandas as pd
import json

from geopy.distance import distance
from shapely.geometry import Point
import matplotlib.pyplot as plt

import cartoframes
from cartoframes.auth import set_default_credentials
from cartoframes.data.observatory import Enrichment
from cartoframes.data.services import Geocoding, Isolines
from cartoframes.viz import Map, Layer, color_continuous_style, size_continuous_style

Preparo cliente para poder hacer queries e ir filtrando el dataset.

In [7]:
from pymongo import MongoClient
client = MongoClient("mongodb://localhost/companies")
db = client.get_database()

Query a MongoDB de empresas que:
- Se hayan creado hace 10 años o menos
- Sigan existiendo
- Dispongan de coordenadas en al menos una de sus oficinas
- Hayan conseguido $1M de inversión o más

In [8]:
year=2010
filtro = {"$and": [{"founded_year":{'$gte':year}},
                   {"deadpooled_year":{"$eq":None}},
                   {"offices":{"$elemMatch":{"latitude":{"$ne":None}}}},
                   {"total_money_raised":{'$gte':"$1000000"}}]}

# Lista que cumple con la query:
new = list(db.companies.find(filtro, projection={"_id":0, "name":1, "offices":1, "founded_year":1}))

# Hago DataFrame de la lista:
new = pd.DataFrame(new)
print("Existen ", len(new), " empresas creadas después de " , year , " cuyas coordenadas están disponibles.")
new

Existen  32  empresas creadas después de  2010  cuyas coordenadas están disponibles.


,name,founded_year,offices
0,GENWI,2010,"[{'description': '', 'address1': '4966 El Cami..."
1,Fixya,2013,"[{'description': '', 'address1': 'One Franklin..."
2,Widgetbox,2012,"[{'description': None, 'address1': None, 'addr..."
3,RazorGator,2011,"[{'description': None, 'address1': '11150 Sant..."
4,Social Gaming Network,2011,"[{'description': 'HQ Los Angeles', 'address1':..."
5,Skydeck,2012,"[{'description': '', 'address1': '330 Waverley..."
6,Magento,2010,"[{'description': None, 'address1': '', 'addres..."
7,Pixsta,2010,"[{'description': None, 'address1': '9 Thorpe C..."
8,FirstString,2011,"[{'description': None, 'address1': '', 'addres..."
9,Titan Gaming,2010,"[{'description': 'Santa Monica Office', 'addre..."


Exploto el dataframe de modo que pueda tener latitud y longitud en columnas independientes:

In [14]:
a = new.explode("offices")
b = a[["offices"]].apply(lambda x: x.offices, result_type="expand", axis=1)
df_new = pd.concat([a,b], axis=1)

# Aumenta el número de registros puesto que algunas compañías tienen varias oficinas.
print(len(new), "--- VS ---" ,len(df_new))
df_new

32 --- VS --- 43


,name,founded_year,offices,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude
0,GENWI,2010,"{'description': '', 'address1': '4966 El Camin...",,4966 El Camino Real,Suite 200,94022,Los Altos,CA,USA,33.817100,-111.903500
1,Fixya,2013,"{'description': '', 'address1': 'One Franklin ...",,One Franklin Parkway,"Building 910, Suite 110",94401,San Mateo,CA,USA,37.566879,-122.323895
2,Widgetbox,2012,"{'description': None, 'address1': None, 'addre...",None,None,None,94111,San Francisco,CA,USA,37.798853,-122.398599
3,RazorGator,2011,"{'description': None, 'address1': '11150 Santa...",None,11150 Santa Monica Blvd. Suite 500,None,90025,Los Angeles,CA,USA,34.047312,-118.445243
4,Social Gaming Network,2011,"{'description': 'HQ Los Angeles', 'address1': ...",HQ Los Angeles,9570 W. Pico Blvd 3rd Floor,,90035,Los Angeles,CA,USA,37.446823,-122.161523
4,Social Gaming Network,2011,"{'description': '', 'address1': '9606 S. Santa...",,9606 S. Santa Monica Blvd,2nd Floor,90210,Beverly Hills,CA,USA,34.069849,-118.405418
5,Skydeck,2012,"{'description': '', 'address1': '330 Waverley ...",,330 Waverley St.,Suite 3,94301,Palo Alto,CA,USA,37.564538,-122.325470
6,Magento,2010,"{'description': None, 'address1': '', 'address...",None,,,,Los Angeles,CA,USA,34.052187,-118.243425
7,Pixsta,2010,"{'description': None, 'address1': '9 Thorpe Cl...",None,9 Thorpe Close,Portobello Rd,W10 5XL,London,None,GBR,51.518190,-0.208979
8,FirstString,2011,"{'description': None, 'address1': '', 'address...",None,,,,Los Angeles,CA,USA,34.052187,-118.243425


Elimino la columna offices y las filas duplicadas:

In [166]:
df = df_new.drop(columns=["offices"])

In [167]:
df.drop_duplicates(inplace=True)

Hay algunos registros que no tienen coordenadas pero sí direcciones postales.
Puedo rellenar esos valores haciendo una request a la API de Google para que me devuelva coordenadas en base a direcciones.
Para eso, primero hago una columna "Address" que contenga en un string la dirección completa para pasarla a la API.

In [173]:
df["Address"] = df.address1 + ", " + df.zip_code + ", " + df.city + ", " + df.country_code

In [174]:
df

,name,founded_year,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude,Address
0,GENWI,2010,,4966 El Camino Real,Suite 200,94022,Los Altos,CA,USA,33.817100,-111.903500,"4966 El Camino Real, 94022, Los Altos, USA"
1,Fixya,2013,,One Franklin Parkway,"Building 910, Suite 110",94401,San Mateo,CA,USA,37.566879,-122.323895,"One Franklin Parkway, 94401, San Mateo, USA"
2,Widgetbox,2012,None,None,None,94111,San Francisco,CA,USA,37.798853,-122.398599,NaN
3,RazorGator,2011,None,11150 Santa Monica Blvd. Suite 500,None,90025,Los Angeles,CA,USA,34.047312,-118.445243,"11150 Santa Monica Blvd. Suite 500, 90025, Los..."
4,Social Gaming Network,2011,HQ Los Angeles,9570 W. Pico Blvd 3rd Floor,,90035,Los Angeles,CA,USA,37.446823,-122.161523,"9570 W. Pico Blvd 3rd Floor, 90035, Los Angele..."
4,Social Gaming Network,2011,,9606 S. Santa Monica Blvd,2nd Floor,90210,Beverly Hills,CA,USA,34.069849,-118.405418,"9606 S. Santa Monica Blvd, 90210, Beverly Hill..."
5,Skydeck,2012,,330 Waverley St.,Suite 3,94301,Palo Alto,CA,USA,37.564538,-122.325470,"330 Waverley St., 94301, Palo Alto, USA"
6,Magento,2010,None,,,,Los Angeles,CA,USA,34.052187,-118.243425,", , Los Angeles, USA"
7,Pixsta,2010,None,9 Thorpe Close,Portobello Rd,W10 5XL,London,None,GBR,51.518190,-0.208979,"9 Thorpe Close, W10 5XL, London, GBR"
8,FirstString,2011,None,,,,Los Angeles,CA,USA,34.052187,-118.243425,", , Los Angeles, USA"


Esto es una alternativa a la API de Google, pero el servicio no está disponible temporalmente:

In [ ]:
from time import sleep 
import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim

def coord(x):
    sleep(1)
    geolocator = Nominatim(user_agent = "myGeocoder")
    location = geolocator.geocode(x)
    if location == None:
        return "not possible"
    else:
        return (location.latitude, location.longitude)

Request a la API de Google:

In [202]:
import requests
import os
from dotenv import load_dotenv

load_dotenv()
gooKey = os.getenv("GoogleAPIKey")
apikey = gooKey

def getCoord(address):
    key = apikey
    url = f'https://maps.googleapis.com/maps/api/geocode/json?address={address}&key={key}'
    response = requests.get(url)
    data = response.json()
    return data
    # me devuelve un diccionario

Hago la request a la API para los casos en los que las coordenadas sean NaN:

In [176]:
data = df[df["latitude"].isna()]["Address"].apply(getCoord)

Compruebo los resultados:
- Me devulve 3 correctos.
- El erróneo es debido seguramente al caracter "&" puesto que si elimino esa parte del string, me devuelve las coordenadas correctas.

In [177]:
data

10    {'results': [{'address_components': [{'long_na...
10    {'results': [{'address_components': [{'long_na...
10    {'results': [{'address_components': [{'long_na...
22            {'results': [], 'status': 'ZERO_RESULTS'}
Name: Address, dtype: object

Con la función **cleanCoord** accedo a la parta del json que contiene las coordenadas:

In [178]:
def cleanCoord(x):
    print(type(x))
    latGoo = x['results'][0]['geometry']['location']['lat']
    lngGoo = x['results'][0]['geometry']['location']['lng']
    return (latGoo, lngGoo)

In [181]:
valores = data[:-1].apply(cleanCoord)
valores = list(valores)
print(valores)

<class 'dict'>
<class 'dict'>
<class 'dict'>
[(39.752908, -104.9965953), (37.78775419999999, -122.4040147), (42.1900278, -71.53822339999999)]


- Para evitar **"index out of range"**, prescindo del último elemento de la lista y relleno los NaN con los nuevos valores.
- Compruebo con un print qué **"Address"** aún sigue teniendo coordenadas NaN.
- Podría volver a hacer una request a la API modificando el string, pero continúo.

In [182]:
addresses = list(df[df["latitude"].isna()]["Address"])
addr = addresses[:-1]

i=0
for a in addr:
    df.loc[df['Address'] == a,"latitude"] = valores[i][0]
    df.loc[df['Address'] == a,"longitude"] = valores[i][1]
    i+=1
    
print(addresses)

['ArcLabs Research & Innovation Building, Waterford, Waterford, IRL']


In [183]:
df2 = df

Para poder continuar con un DataFrame limpio, elimino la fila con latitud = longitud = NaN

In [185]:
df2.dropna(subset=["latitude","longitude"], inplace=True)

In [187]:
df2

,name,founded_year,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude,Address
0,GENWI,2010,,4966 El Camino Real,Suite 200,94022,Los Altos,CA,USA,33.817100,-111.903500,"4966 El Camino Real, 94022, Los Altos, USA"
1,Fixya,2013,,One Franklin Parkway,"Building 910, Suite 110",94401,San Mateo,CA,USA,37.566879,-122.323895,"One Franklin Parkway, 94401, San Mateo, USA"
2,Widgetbox,2012,None,None,None,94111,San Francisco,CA,USA,37.798853,-122.398599,NaN
3,RazorGator,2011,None,11150 Santa Monica Blvd. Suite 500,None,90025,Los Angeles,CA,USA,34.047312,-118.445243,"11150 Santa Monica Blvd. Suite 500, 90025, Los..."
4,Social Gaming Network,2011,HQ Los Angeles,9570 W. Pico Blvd 3rd Floor,,90035,Los Angeles,CA,USA,37.446823,-122.161523,"9570 W. Pico Blvd 3rd Floor, 90035, Los Angele..."
4,Social Gaming Network,2011,,9606 S. Santa Monica Blvd,2nd Floor,90210,Beverly Hills,CA,USA,34.069849,-118.405418,"9606 S. Santa Monica Blvd, 90210, Beverly Hill..."
5,Skydeck,2012,,330 Waverley St.,Suite 3,94301,Palo Alto,CA,USA,37.564538,-122.325470,"330 Waverley St., 94301, Palo Alto, USA"
6,Magento,2010,None,,,,Los Angeles,CA,USA,34.052187,-118.243425,", , Los Angeles, USA"
7,Pixsta,2010,None,9 Thorpe Close,Portobello Rd,W10 5XL,London,None,GBR,51.518190,-0.208979,"9 Thorpe Close, W10 5XL, London, GBR"
8,FirstString,2011,None,,,,Los Angeles,CA,USA,34.052187,-118.243425,", , Los Angeles, USA"


In [189]:
df3 = df2.to_json("../output/test2.json", orient = "records")

In [205]:
import geopandas as gpd

gdf = gpd.GeoDataFrame(df3, geometry = gpd.points_from_xy(df.longitude, df.latitude))
l2 = gdf
print(f'Tipo: {type(gdf)}')
l2

Tipo: <class 'geopandas.geodataframe.GeoDataFrame'>


,geometry
0,POINT (-111.90350 33.81710)
1,POINT (-122.32389 37.56688)
2,POINT (-122.39860 37.79885)
3,POINT (-118.44524 34.04731)
4,POINT (-122.16152 37.44682)
5,POINT (-118.40542 34.06985)
6,POINT (-122.32547 37.56454)
7,POINT (-118.24343 34.05219)
8,POINT (-0.20898 51.51819)
9,POINT (-118.24343 34.05219)


Mapa de Carto con los puntos del geoDataFrame:

In [195]:
Map(Layer(l2))

Request a la API de FourSquare:

In [204]:
import json, requests

FourSquareID = os.getenv("FSID")
fsID = FourSquareID

FourSquareKEY = os.getenv("FSKEY")
fsKEY = FourSquareKEY

url = 'https://api.foursquare.com/v2/venues/search'
params = dict(
  client_id='fsID',
  client_secret='fsKEY',
  v='20200211',
  ll='40.7243,-74.0018',
  query='airport',
  limit=1
)
resp = requests.get(url=url, params=params)
data = json.loads(resp.text)